# Utilisation API Pôle emploi
Source : https://github.com/etiennekintzler/api-offres-emploi

In [15]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import HTTPError

import datetime
import time
import re

In [16]:
client_id="PAR_textmining_6361ebc3d9de749cdfb72158572fa975a0ba30e97e4e570957f187132bb2b361"
client_secret="d77c395663a600cb89716cd8a6c02ab9b2d6920fbdc520791bcff93578b2f093"

In [17]:
def get_token(client_id,client_secret):
    data = dict(
                grant_type="client_credentials",
                client_id=client_id,
                client_secret=client_secret,
                scope="api_offresdemploiv2 o2dsoffre application_{}".format(client_id),
            )

    headers = {"content-type": "application/x-www-form-urlencoded"}
    params = dict(realm="/partenaire")
    current_time = datetime.datetime.today()
    r = requests.post(
        url="https://entreprise.pole-emploi.fr/connexion/oauth2/access_token",
        headers=headers,
        data=data,
        params=params,
        timeout=60
            )
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + str(r.json())
            raise HTTPError(complete_message)
        else:
            raise error
    else:
        token = r.json()
        token["expires_at"] = current_time + datetime.timedelta(
            seconds=token["expires_in"]
        )
        
        return token

In [18]:
token=get_token(client_id,client_secret)
token

{'access_token': 'phC_uzPPQt9VZqK0w1irPOYBIFU',
 'scope': 'application_PAR_textmining_6361ebc3d9de749cdfb72158572fa975a0ba30e97e4e570957f187132bb2b361 api_offresdemploiv2 o2dsoffre',
 'token_type': 'Bearer',
 'expires_in': 1499,
 'expires_at': datetime.datetime(2023, 12, 14, 21, 14, 31, 496591)}

In [19]:
def get_hearders(token):
    headers = {
            "Authorization": "Bearer {}".format(token["access_token"])
        }
    return headers


In [20]:
# Valeurs constantes

OFFRES_DEMPLOI_V2_BASE = "https://api.emploi-store.fr/partenaire/offresdemploi/v2"
REFERENTIEL_ENDPOINT = "{}/referentiel".format(OFFRES_DEMPLOI_V2_BASE)
SEARCH_ENDPOINT = "{}/offres/search".format(OFFRES_DEMPLOI_V2_BASE)

In [21]:
# 'Réferentiel' available: domaine, appellations (domaines professionnelles ROME), metiers, themes, continents,
#         pays, regions, departements , communes , secteursActivites, naturesContrats,  typesContrats, niveauxFormations,
#         permis, langues

def referentiel(ref):
    
    r = requests.Session().get(
                url="{}/{}".format(REFERENTIEL_ENDPOINT, ref),
                params= dict(realm="/partenaire"),
                headers=get_hearders()
            )

    try:
        r.raise_for_status()
    except Exception as e:
        raise e
    else:
        return r.json()

In [22]:
def search(params=None):
    print('Requête avec les params {}'.format(params))
    r = requests.Session().get(
            url=SEARCH_ENDPOINT,
            params=params,
            headers=get_hearders(token)
        )

    silent_http_errors=False
    try:
        r.raise_for_status()
    except HTTPError as error:
        if r.status_code == 400:
            complete_message = str(error) + "\n" + r.json()["message"]
            if silent_http_errors:
                print(complete_message)
            else:
                raise HTTPError(complete_message)
        else:
            if silent_http_errors:
                print(str(error))
            else:
                raise error
    else:
        found_range = re.search(
            pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",
            string=r.headers["Content-Range"],
        ).groupdict()
        out = r.json()
        out.update({"Content-Range": found_range})
        return out

<>:26: SyntaxWarning: invalid escape sequence '\d'
<>:26: SyntaxWarning: invalid escape sequence '\d'
C:\Users\barai\AppData\Local\Temp\ipykernel_19016\1347239471.py:26: SyntaxWarning: invalid escape sequence '\d'
  pattern="offres (?P<first_index>\d+)-(?P<last_index>\d+)/(?P<max_results>\d+)",


# Exemple

In [23]:
start_dt = datetime.datetime(2020, 3, 1, 12, 30)
end_dt = datetime.datetime.today()
params = {

    "motsCles": "data science",
    'minCreationDate': start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"),
    'maxCreationDate': end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
}

ex_search=search(params)


Requête avec les params {'motsCles': 'data science', 'minCreationDate': '2020-03-01T12:30:00Z', 'maxCreationDate': '2023-12-14T20:49:43Z'}


In [26]:
ex_search

AttributeError: 'dict' object has no attribute 'dtype'

In [25]:
import sqlite3

In [7]:
conn = sqlite3.connect('job_mining.db')

In [8]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE LieuTravail_Dimension (
    id TEXT PRIMARY KEY,
    libelle TEXT,
    latitude REAL,
    longitude REAL,
    codePostal TEXT,
    commune TEXT
)
''')



OperationalError: table LieuTravail_Dimension already exists

In [9]:

cursor.execute('''
CREATE TABLE Entreprise_Dimension (
    id TEXT PRIMARY KEY,
    nom TEXT,
    adaptee INTEGER
)
''')


In [10]:
cursor.execute('''
CREATE TABLE Contrat_Dimension (
    id TEXT PRIMARY KEY,
    typeContrat TEXT,
    typeContratLibelle TEXT,
    natureContrat TEXT,
    dureeTravailLibelle TEXT,
    dureeTravailLibelleConverti TEXT,
    alternance INTEGER
)
''')

In [11]:
cursor.execute('''
    CREATE TABLE OrigineOffre_Dimension (
        id TEXT PRIMARY KEY,
        origine TEXT,
        urlOrigine TEXT
    )
''')

In [12]:
# Création de la table de faits (OffresEmploi_Faits)
cursor.execute('''
    CREATE TABLE OffresEmploi_Faits (
        id TEXT PRIMARY KEY,
        dateCreation TEXT,
        dateActualisation TEXT,
        nombrePostes INTEGER,
        salaireLibelle TEXT,
        lieuTravailId TEXT REFERENCES LieuTravail_Dimension(id),
        entrepriseId TEXT REFERENCES Entreprise_Dimension(id),
        contratId TEXT REFERENCES Contrat_Dimension(id),
        qualificationId TEXT REFERENCES Qualification_Dimension(id),
        origineOffreId TEXT REFERENCES OrigineOffre_Dimension(id)
    )
''')

In [13]:
# Commit et fermeture de la connexion
conn.commit()
conn.close()

In [14]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
conn = sqlite3.connect('job_mining.db')

# Exécuter une requête SQL (remplacez la requête par celle que vous souhaitez)
query = "SELECT * FROM OffresEmploi_Faits"
df = pd.read_sql_query(query, conn)

# Afficher le DataFrame
print(df)

# Fermer la connexion
conn.close()

Empty DataFrame
Columns: [id, dateCreation, dateActualisation, nombrePostes, salaireLibelle, lieuTravailId, entrepriseId, contratId, qualificationId, origineOffreId]
Index: []
